# DXF to PNG Converter (Google Colab Edition)

このノートブックは、DXFファイルをPNG形式に変換します。日本語テキストの表示もサポートしています。

This notebook converts DXF files to PNG format with Japanese text support.

## 1. 環境セットアップ / Environment Setup

必要なパッケージとフォントをインストールします。

In [ ]:
# Install required packages
!pip install ezdxf matplotlib pillow

# Install Japanese fonts (for Google Colab)
import os
if 'COLAB_GPU' in os.environ or 'COLAB_TPU_ADDR' in os.environ:
    print("Installing Japanese fonts for Google Colab...")
    !apt-get -y install fonts-noto-cjk fonts-ipafont-gothic fonts-ipafont-mincho
    print("✓ Japanese fonts installed")
else:
    print("Not running in Google Colab. Please ensure Japanese fonts are installed on your system.")
    print("macOS: brew install font-noto-sans-cjk-jp")
    print("Linux: sudo apt-get install fonts-noto-cjk")

## 2. ライブラリのインポートとフォント設定 / Import Libraries and Setup Fonts

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import ezdxf
from ezdxf import recover
from ezdxf.addons.drawing import matplotlib as ezdxf_matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from PIL import Image
from pathlib import Path
import platform

# Clear font cache and rebuild
fm._rebuild()

# Platform detection and font setup
system = platform.system()
print(f"Running on: {system}")

# Find available Japanese fonts
japanese_fonts = []
font_priority = [
    "Noto Sans CJK JP", "Hiragino Sans", "Yu Gothic", "IPAGothic", 
    "Hiragino Kaku Gothic Pro", "Osaka", "Meiryo", "MS Gothic"
]

available_fonts = [f.name for f in fm.fontManager.ttflist]
for font in font_priority:
    if font in available_fonts:
        japanese_fonts.append(font)

if japanese_fonts:
    selected_font = japanese_fonts[0]
    plt.rcParams['font.family'] = selected_font
    print(f"✓ Using Japanese font: {selected_font}")
else:
    print("⚠️  No Japanese fonts found. Text may not display correctly.")
    print("Available fonts:", available_fonts[:10], "...")

## 3. DXFファイルのアップロード / Upload DXF Files

DXFファイルをアップロードしてください。

In [ ]:
from google.colab import files
import os

# Create upload directory
upload_dir = "uploaded_dxf"
os.makedirs(upload_dir, exist_ok=True)

print("DXFファイルをアップロードしてください / Please upload your DXF files:")
uploaded = files.upload()

# Save uploaded files
for filename, content in uploaded.items():
    filepath = os.path.join(upload_dir, filename)
    with open(filepath, 'wb') as f:
        f.write(content)
    print(f"✓ Uploaded: {filename}")

# List uploaded files
dxf_files = list(Path(upload_dir).glob("*.dxf"))
print(f"\n{len(dxf_files)} DXF file(s) uploaded")

## 4. DXF to PNG 変換関数 / Conversion Function

In [ ]:
def convert_dxf_to_png(dxf_path, png_path, dpi=300):
    """Convert DXF file to PNG with Japanese text support"""
    try:
        # Load DXF file
        print(f"Loading: {dxf_path}")
        try:
            doc = ezdxf.readfile(dxf_path)
        except Exception as e:
            print(f"Standard loading failed: {e}")
            print("Attempting recovery mode...")
            doc, auditor = recover.readfile(dxf_path)
            if auditor.has_errors:
                print(f"Recovery completed with {len(auditor.errors)} errors")
        
        # Create figure
        fig = plt.figure(figsize=(12, 9))
        ax = fig.add_axes([0, 0, 1, 1])
        
        # Draw DXF
        msp = doc.modelspace()
        backend = ezdxf_matplotlib.MatplotlibBackend(ax)
        frontend = ezdxf.addons.drawing.Frontend(backend)
        frontend.draw_layout(msp, finalize=True)
        
        # Set aspect ratio
        ax.set_aspect('equal')
        
        # Save PNG
        print(f"Saving: {png_path}")
        fig.savefig(png_path, dpi=dpi, bbox_inches='tight', pad_inches=0.1)
        plt.close(fig)
        
        # Verify
        if os.path.exists(png_path):
            img = Image.open(png_path)
            print(f"✓ Success: {img.size[0]}x{img.size[1]} pixels")
            return True
        else:
            print("✗ Failed: Output file not created")
            return False
            
    except Exception as e:
        print(f"✗ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

## 5. 変換実行 / Execute Conversion

In [ ]:
# Create output directory
output_dir = "converted_png"
os.makedirs(output_dir, exist_ok=True)

# Convert all DXF files
converted_files = []

for dxf_file in dxf_files:
    png_file = Path(output_dir) / f"{dxf_file.stem}.png"
    print(f"\nConverting: {dxf_file.name}")
    
    if convert_dxf_to_png(str(dxf_file), str(png_file), dpi=300):
        converted_files.append(png_file)

print(f"\n✓ Converted {len(converted_files)} file(s)")

## 6. 結果の表示 / Display Results

In [ ]:
# Display converted images
from IPython.display import Image as IPImage, display

for png_file in converted_files:
    print(f"\n📄 {png_file.name}:")
    display(IPImage(str(png_file), width=600))

## 7. ダウンロード / Download Converted Files

In [ ]:
# Download all converted PNG files
print("Downloading converted files...")

for png_file in converted_files:
    print(f"Downloading: {png_file.name}")
    files.download(str(png_file))

print("\n✓ All files downloaded!")

## トラブルシューティング / Troubleshooting

### 日本語が表示されない場合 / If Japanese text doesn't display:

1. **フォントの確認 / Check fonts:**

In [ ]:
# Check available Japanese fonts
print("Available fonts with CJK support:")
cjk_fonts = []
for font in fm.fontManager.ttflist:
    if any(term in font.name.lower() for term in ['cjk', 'japan', 'gothic', 'mincho', 'hiragino', 'noto']):
        cjk_fonts.append(font.name)

for font in sorted(set(cjk_fonts)):
    print(f"  - {font}")

if not cjk_fonts:
    print("  No CJK fonts found!")
    print("\nFor macOS users:")
    print("  1. Install via Homebrew: brew install font-noto-sans-cjk-jp")
    print("  2. Or download from: https://www.google.com/get/noto/")
    print("\nFor local notebook (not Colab):")
    print("  Restart kernel after installing fonts")

### macOS固有の問題 / macOS Specific Issues:

macOSでフォントが見つからない場合:

1. **Homebrewでインストール:**
   ```bash
   brew install font-noto-sans-cjk-jp
   ```

2. **Font Bookでシステムフォントを確認:**
   - Applications → Font Book
   - 「日本語」カテゴリを確認

3. **Microsoft Officeがインストールされている場合:**
   - MS Gothic, Meiryoなどが利用可能

4. **カーネルを再起動:**
   - フォントインストール後は必ずJupyterカーネルを再起動